# Final Project Geoscripting
**Team: Imperfect_GeoCircle 
**Authors: Yanuar Bomantara & Thijs Seubring 
**Date: 28-01-2021

## Introduction
Wildfires are happening more and more around the world. Information about these fires is key for the safety of people. We want to create a program that gives information on past fires. This program will provide valuable insight in land cover that is affected by wildfires. The program can be used anywhere on the world, so anyone concerned by forest fires can use it.

The burned area data is provided by ESA and is retrieved from from GEE using the python API. More information on this data-set can be found [here.][https://developers.google.com/earth-engine/datasets/catalog/ESA_CCI_FireCCI_5_1]. The user defines their own AOI by drawing a vector on the voila map.

To start this program as an interactive web apps please click Voila button in the top bar of this notebook.

## Calculating Burned Area

**Steps to create a Burned Area Map:**

1. Pan and zoom to your region of interest.
2. Use the drawing tool to draw a rectangle anywhere on the map.
3. Adjust the parameters (start year and end year).
4. Click the 'Submit AOI' button to create Area of Interest.
5. Click the 'Submit' button to load burned area into map (wait until the notification process finished).
6. Generate chart by clicking 'Generate Chart' button.

In [ ]:
# Backup command just in case you need it

#conda install geemap
#conda install ee
#conda install rasterio

#ee.Authenticate()
#ee. Initialize()

## Make sure to put this cell into comment again after used it before you execute Voila

In [ ]:
#Importing Library
import os
import ee
import geemap
import gdal
import ipywidgets as widgets
from IPython.display import display
import numpy
import matplotlib.pyplot as plt

In [ ]:
#Importing Function
import GetBurnedArea as functs

In [ ]:
#Load the basic map from geemap library

Map = geemap.Map()
Map

In [ ]:
#Create legend for the Map

legend_dict = {
    'Cropland': (237,167,14),
    'Forest': (8, 133, 16),
    'Shrubland': (14, 176, 127),
    'Grassland' : (178, 224, 135)
}
Map.add_legend(legend_title='Land Cover', legend_dict=legend_dict, position='bottomright')

In [ ]:
#Define Map Style

style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Burned Area',
    width=200,
    style=style
)

In [ ]:
# Define the AOI and pass it to widget button 

feature=None
roi=None

refresh = widgets.Button(
    description='Submit AOI',
    button_style='primary',
    tooltip='Click to submit Area of Interest from your draw',
    style=style
)

def refresh_clicked(b):
    global feature
    global roi
    feature = Map.draw_last_feature
    roi = feature.geometry()
    
refresh.on_click(refresh_clicked)

In [ ]:
# Create directory

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# Define start year variable and pass it to slider widget
start_year = widgets.IntSlider(description='Start Year:', value=20001, min=2001, max=2019, style=style)

In [ ]:
# Define end year variable and pass it to slider widget
end_year = widgets.IntSlider(description='End Year:', value=2019, min=2001, max=2019, style=style)

In [ ]:
# Define slider widget
hbox1 = widgets.HBox([start_year, end_year])
hbox1

In [ ]:
# Define Submint button widget

output = widgets.Output()

BurnedArea={}

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click the submit to execute',
    style=style
)

In [ ]:
# Demonstrate AddMap function to add the result maps on the map and pass it to submit button widget

def AddMap(b) :
    for i in range(start_year.value, end_year.value +1):
        startdate = str(i)+'-01-01'
        enddate = str(i)+'-12-31'
        baVis = {
            'min': 10,
            'max': 180,
            'palette': [
                'eda70e', 'eda70e', 'eda70e', '0eb07f', '088510', '088510',
                '088510', '088510', '088510', '088510', '088510', '0eb07f',
                'b2e087', 'b2e087', 'b2e087', '088510', '0eb07f',
            ]
        }

        dataset = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate(str(startdate), str(enddate))
        burnedArea = dataset.select('LandCover')
        maxBA = burnedArea.max()
        maxBAclip = maxBA.clip(feature)
        Map.addLayer(maxBAclip, baVis, str(i)+' Burned Area')
            
            

submit.on_click(AddMap)

In [ ]:
# Demonstrate function to download maps data from each year and make dictionary that save surface area of burned area

def submit_clicked(b):
    with output:
        output.clear_output()
        if start_year.value > end_year.value:
            print('The end year must be great than the start year.')
        
        global BurnedArea
        BurnedArea = {}
        for i in range(start_year.value, end_year.value +1):
            BAyear = functs.GetBurnedArea(i,feature)
            BurnedArea[i]= BAyear
        return(BurnedArea)
            
submit.on_click(submit_clicked)

In [ ]:
# Variable to pass Submit AOI button widget
refresh

In [ ]:
# Variable to pass Submit button widget
submit

In [ ]:
# Demonstrate function to plot the histogram from the BurnedArea dictionary and pass it to Generate Chart button widget.

from ipywidgets.widgets.interaction import show_inline_matplotlib_plots

chart = widgets.Button(
    description='Generate Chart',
    button_style='primary',
    tooltip='Click the submit to generate chart',
    style=style
)

def chart_clicked(b):
    with output :
        output.clear_output()
        global BurnedArea
        functs.BurntAreaPlot(BurnedArea)
        show_inline_matplotlib_plots()
    
chart.on_click(chart_clicked)

In [ ]:
# Variable to pass Generate button widget
chart

In [ ]:
output